In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [2]:
movies = pd.read_csv("data/movies.csv").sample(frac=1, random_state=51)

In [3]:
movies.head()

,Title,Director,Cast,Genre,Combined
59,Kakanfo,David Dida Tella,Bimbo Oshin Antar Laniyan Dele Odule,Adventure Drama,Bimbo Oshin Antar Laniyan Dele Odule David Did...
64,Citation,Kunle Afolayan,Temi Otedola Jimmy Jean-Lewis Kunle Afolayan J...,Drama Thriller,Temi Otedola Jimmy Jean-Lewis Kunle Afolayan J...
123,King of Thieves (Ogundabede),Adebayo Tijani and Tope Adebayo Salami,Toyin Abraham Femi Adebayo Salami Odunlade Ade...,Drama,Toyin Abraham Femi Adebayo Salami Odunlade Ade...
99,Mimi,Samuel Olatunji,Ali Baba Ireti Doyle Toyin Abraham Prince Jide...,Romance Comedy,Ali Baba Ireti Doyle Toyin Abraham Prince Jide...
58,Fate of Alakada: The Party Planner,Kayode Kasum,Toyin Abraham Mercy Eke Broda Shaggi,Action Comedy,Toyin Abraham Mercy Eke Broda Shaggi Kayode Ka...


In [4]:
tfv = TfidfVectorizer(min_df=3, strip_accents="unicode", analyzer="word", ngram_range=(1, 3), stop_words="english")

In [5]:
count_matrix = tfv.fit_transform(movies['Combined'])

In [6]:
similarity = sigmoid_kernel(count_matrix, count_matrix)

In [7]:
import joblib
joblib.dump(similarity, "similarity_joblib")

['similarity_jobllib']

In [11]:
def recommend(title):
    index = movies.loc[movies["Title"] == title].index[0]
    similiar_movies = list(enumerate(similarity[index]))
    similiar_movies_index = sorted(similiar_movies, key=lambda x:x[1], reverse=True)
    movies_index = similiar_movies_index[1:11]
    
    movie = []
    for i in range(len(movies_index)):
        movie_name = movies_index[i][0]
        movie.append(movies["Title"][movie_name])
    return movie

In [12]:
def recommend(title):
    index = movies.loc[movies["Title"] == title].index[0]
    similiar_movies = list(enumerate(similarity[index]))
    similiar_movies_index = sorted(similiar_movies, key=lambda x:x[1], reverse=True)
    movies_index = similiar_movies_index[1:11]


    movie = []
    poster = []
    for i in range(len(movies_index)):
        movie_name = movies_index[i][0]
        name = movies["Title"][movie_name]
        from tmdbv3api import Movie, TMDb
        tmdb = TMDb()
        tmdb.api_key = 'API HERE'
        tmdb_movie = Movie()
        try:
            result = tmdb_movie.search(name)
            movie_id = result[0].id
        except:
            movie_id = None
        movie.append(movies["Title"][movie_name])
        poster.append(movie_id)
    return movie, poster

In [13]:
test = recommend("Quam's Money")

In [14]:
test

(['Nneka the pretty serpent',
  'The Cock-Tale',
  'Chief Daddy',
  'King of Boys',
  'Rattlesnake: The Ahanna story',
  'Bling Lagosians',
  'Obey',
  'Prophetess',
  'Sylvia',
  'The Ghost and the Tout Too'],
 [862280, None, 922017, 570465, 800415, 638691, 18002, 817203, 28200, 1004252])